# Load 

In [2]:
import os

dataset = ''
files = os.listdir('../data/cleaned-data/')
for file_name in files:
    with open(f'../data/cleaned-data/{file_name}', 'r', encoding='utf-8') as f:
        dataset += f.read() + '\n\n'

# Split

### Semantic chunking with each article

In [3]:
chunks = dataset.split('\n\n')
len(chunks)

238

### Split by characters

In [9]:
from langchain_text_splitters import CharacterTextSplitter

chunk_size = 1000
chunk_overlap = 0.2 * chunk_size

text_splitter = CharacterTextSplitter(
    separator='',
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)
chunks = text_splitter.split_text(dataset)
len(chunks)

302

### Summary

In [ ]:
# import time
# from tqdm import tqdm
# from google import genai

# client = genai.Client(api_key='')
# prompt = '''
# **دستور العمل:**
# ماده زیر را از آیین‌نامه‌های دانشگاه ولی عصر رفسنجان به طور کامل و دقیق خلاصه کنید، به گونه‌ای که تنها نکات کلیدی و اطلاعات ضروری حفظ شوند و برای استفاده در یک سیستم RAG (Retrieval-Augmented Generation) مناسب باشد. از هرگونه توضیحات اضافی یا تفسیر خودداری کنید و فقط به متن اصلی خلاصه شده بسنده نمایید.

# **مثال:**
# ماده ۱۲: دانشجویان موظفند کلیه تکالیف درسی خود را در مهلت مقرر تحویل نمایند. عدم تحویل تکالیف یا تأخیر در آن می‌تواند منجر به کسر نمره یا عدم پذیرش تکلیف گردد. در صورت وجود دلایل موجه برای تأخیر، دانشجو باید مراتب را به اطلاع استاد مربوطه برساند و با ارائه مستندات، درخواست تمدید مهلت نماید. تصمیم‌گیری نهایی در خصوص پذیرش تأخیر بر عهده استاد درس است.
# خروجی مورد انتظار از مدل (برای مثال بالا):
# "دانشجویان باید تکالیف را به موقع تحویل دهند. تأخیر یا عدم تحویل ممکن است به کسر نمره یا عدم پذیرش منجر شود. در صورت دلایل موجه برای تأخیر، دانشجو باید با استاد هماهنگ کرده و مستندات ارائه دهد. پذیرش تأخیر با استاد است." 

# **متن ماده:** 
# {context}

# **خلاصه ماده:**
# '''.strip()

# result = []

# for data in tqdm(dataset):
#     response = client.models.generate_content(
#         model="gemini-2.5-flash-lite",
#         contents=prompt.format(context=data),
#     )
#     result.append(response.text)
#     time.sleep(4)

# with open('output.txt', 'w', encoding='utf-8') as f:
#     f.write('\n---\n'.join(result))

In [15]:
with open('../data/structured/summary.txt', 'r', encoding='utf-8') as f:
    chunks = f.read().split('\n---\n')

### Key sentences

In [19]:
# prompt = '''
# **از شما می‌خواهم که جملات کلیدی و اصلی را از متن آیین‌نامه دانشگاه که در ادامه ارائه می‌شود، استخراج کنید. لطفاً جملات را دقیقاً همانطور که هستند، بدون هیچ‌گونه تغییری، بازگردانید. اگر بیش از یک جمله کلیدی وجود داشت، هر جمله را با `\n\n` از جمله بعدی جدا کنید تا خروجی ساختاریافته باشد.**

# **متن آیین‌نامه:**
# {context}
# '''.strip()

In [20]:
with open('../data/structured/key-sentences.txt', 'r', encoding='utf-8') as f:
    chunks = f.read().split('\n---\n')
chunks = '\n\n'.join(chunks).split('\n\n')    

# Store

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

DEVICE = 'cuda'

vector_store = FAISS.from_texts(
    texts=chunks,
    embedding=HuggingFaceEmbeddings(
        model_name='../config/embedding_model/',
        model_kwargs={'device': DEVICE},
    ),
)

In [22]:
q = 'حداکثر غیبت مجاز برای دانشجویان چگونه است؟'
for x in vector_store.similarity_search(q):
    print(x.page_content, '\n\n')

به پایان رسیدن سقف سنوات مجاز تحصیل دانشجو 


ماده 13: مدت مجاز تحصیل در دوره‌های تحصیلی در دوره کارشناسی پیوسته 8 نیمسال است.
تبصره 1: د صورتی که سنوات دانشجو افزایش یابد با تایید شورای آموزشی دانشگاه، دانشجو در مقطع کارشناسی پیوسته حداکثر از 2 نیمسال سنوات ارفاقی بهره‌مند می‌شود.
تبصره 2: دانشجو باید دلایل کافی مبنی بر علت طولانی شدن تحصیل را به شورای آموزشی دانشگاه اعلام نماید.
تبصره 5: حداقل مدت مجاز برای دانش آموختگی در دوره کارشناسی پیوسته 6 نیمسال تحصیلی می‌باشد. 


ظرفیت پذیرش دانشجوی تحصیلات ت کمیلی (در صورت وج ود دانشجو) عضو نیمه - حضوری نصف عضو تماموقت محاسبه می‌شود. 


تبصره 1: حداکثر واحد درسی قابل اخذ در هر نیمسال تحصیلی 20 واحد و در بازه تابستان 6 واحد است. 




In [ ]:
# vector_store.save_local('../data/knowledge-base/article/')
# vector_store.save_local('../data/knowledge-base/character/')
# vector_store.save_local('../data/knowledge-base/summary/')
vector_store.save_local('../data/knowledge-base/key-sentences/')